In [0]:
adls_account_name = "datalakedemome" 
adls_container_name = "lake" 
adls_mount_point = "/mnt/employeedata"

configs = {
    f"fs.azure.account.key.{adls_account_name}.blob.core.windows.net": "eSxC/r4ljns4ZizgMVFsy4tU+/eUCRdr9pOfJ1fpPtdrYfVcbSUPNgg17cqFtKugewjaI3NavGl2+AStpboORw=="
}

dbutils.fs.mount(
    source=f"wasbs://{adls_container_name}@{adls_account_name}.blob.core.windows.net/",
    mount_point=adls_mount_point,
    extra_configs=configs
)

dbutils.fs.ls(adls_mount_point)

[FileInfo(path='dbfs:/mnt/employeedata/dbo.Employee.csv', name='dbo.Employee.csv', size=66254, modificationTime=1722074169000)]

In [0]:
df = spark.read.csv(adls_mount_point, inferSchema = True, header = True)

In [0]:
df.show()

+---+---------+----------+--------------------+------+------+-----------+
| id|    ename|  hiredate|          department|salary|gender|    country|
+---+---------+----------+--------------------+------+------+-----------+
|  1|   Waylon|2021-03-29|             Support| 76455|  Male|      China|
|  2|   Ailene|2021-03-15|           Marketing| 59224|  Male|Philippines|
|  3|  Bernete|2021-01-10|         Engineering| 65091|Female|      Egypt|
|  4|   Xylina|2021-02-27|          Accounting| 36645|Female|   Thailand|
|  5| Marcello|2020-09-20|Business Development| 32546|  Male|   Colombia|
|  6|  Tamqrah|2020-10-04|Business Development| 32379|  Male|   Colombia|
|  7|     Niel|2020-10-19|           Marketing| 81266|Female|    Finland|
|  8|     Mina|2020-10-12|  Product Management| 69214|Female| Madagascar|
|  9| Annelise|2021-02-04|Business Development| 44301|Female|  Indonesia|
| 10|      Ame|2020-09-04|  Product Management| 96445|Female|     Russia|
| 11|Claudetta|2020-12-27|            

In [0]:
from pyspark.sql.functions import *
new_df = df.withColumn("Updated_Salary", df.salary + 1000).withColumn("hiredate", to_date(df.month_id, 'yyyy-MM-dd'))

In [0]:
new_df = new_df.withColumn("YearID",year(hiredate))
new_df.show()

+---+---------+----------+--------------------+------+------+-----------+-------------+
| id|    ename|  hiredate|          department|salary|gender|    country|Apdated_salry|
+---+---------+----------+--------------------+------+------+-----------+-------------+
|  1|   Waylon|2021-03-29|             Support| 76455|  Male|      China|        77455|
|  2|   Ailene|2021-03-15|           Marketing| 59224|  Male|Philippines|        60224|
|  3|  Bernete|2021-01-10|         Engineering| 65091|Female|      Egypt|        66091|
|  4|   Xylina|2021-02-27|          Accounting| 36645|Female|   Thailand|        37645|
|  5| Marcello|2020-09-20|Business Development| 32546|  Male|   Colombia|        33546|
|  6|  Tamqrah|2020-10-04|Business Development| 32379|  Male|   Colombia|        33379|
|  7|     Niel|2020-10-19|           Marketing| 81266|Female|    Finland|        82266|
|  8|     Mina|2020-10-12|  Product Management| 69214|Female| Madagascar|        70214|
|  9| Annelise|2021-02-04|Busine

In [0]:
# Define storage account and container details for ADLS
adls_storage_account_name = "datalakedemome"
adls_container_name = "newcont"  
output_path = f"abfss://{adls_container_name}@{adls_storage_account_name}.dfs.core.windows.net/transformed_data"
 
spark.conf.set(
    f"fs.azure.account.key.{adls_storage_account_name}.dfs.core.windows.net",
    "eSxC/r4ljns4ZizgMVFsy4tU+/eUCRdr9pOfJ1fpPtdrYfVcbSUPNgg17cqFtKugewjaI3NavGl2+AStpboORw=="
)

df.write.mode('overwrite').option("header", "true").csv(output_path)

print(f"Transformed data to {output_path}")

Transformed data written to abfss://newcont@datalakedemome.dfs.core.windows.net/transformed_data
